## Deep Learning Models

In [7]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please ensure you have installed TensorFlow correctly')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.13.1
Default GPU Device: /device:GPU:0


In [4]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('C:/Users/hims1/Ashrae'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:/Users/hims1/Ashrae\1_eda.ipynb
C:/Users/hims1/Ashrae\Ashrae - Great Energy Predictor III.ipynb
C:/Users/hims1/Ashrae\Ashrae Final  GCP v2 Final - Great Energy Predictor III.ipynb
C:/Users/hims1/Ashrae\Ashrae Final  GCP v2- Great Energy Predictor III.ipynb
C:/Users/hims1/Ashrae\Ashrae Final - Great Energy Predictor III.ipynb
C:/Users/hims1/Ashrae\ashrae.png
C:/Users/hims1/Ashrae\building_metadata.csv
C:/Users/hims1/Ashrae\D2_pred.csv
C:/Users/hims1/Ashrae\final_xte.csv
C:/Users/hims1/Ashrae\sample_submission.csv
C:/Users/hims1/Ashrae\test.csv
C:/Users/hims1/Ashrae\test_pred.csv
C:/Users/hims1/Ashrae\train.csv
C:/Users/hims1/Ashrae\Untitled.ipynb
C:/Users/hims1/Ashrae\Untitled1.ipynb
C:/Users/hims1/Ashrae\Untitled2.ipynb
C:/Users/hims1/Ashrae\weather_test.csv
C:/Users/hims1/Ashrae\weather_train.csv
C:/Users/hims1/Ashrae\X_test.csv
C:/Users/hims1/Ashrae\X_train.csv
C:/Users/hims1/Ashrae\.ipynb_checkpoints\1_eda-checkpoint.ipynb
C:/Users/hims1/Ashrae\.ipynb_checkpoints\Ashrae - Great En

In [46]:
from keras.models import Model, load_model
from keras.layers import Input, Dropout, Dense, Embedding, SpatialDropout1D, concatenate, BatchNormalization, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.models import Model
from keras.losses import mean_squared_error as mse_loss


from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation, Dropout
from keras import optimizers
from keras.optimizers import RMSprop, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

#### Preparing Train & Test Set for fitting into the Model

In [32]:
x_train = pd.read_csv("X_train.csv")

In [33]:
x_test = pd.read_csv("C:/Users/hims1/Ashrae/X_test.csv")

In [34]:
x_train.drop(x_train.columns[x_train.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [36]:
x_test.drop(x_test.columns[x_test.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [38]:
target = np.log1p(x_train["meter_reading"])

In [39]:
target_test = np.log1p(x_test["meter_reading"])

In [41]:
x_train = x_train.drop(columns=['meter_reading', 'meter_reading_log1p'])

In [42]:
x_test = x_test.drop(columns=['meter_reading', 'meter_reading_log1p'])

In [43]:
x_train.head()

,building_id,meter,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,...,wind_direction_max_lag72,wind_speed_max_lag72,wind_speed_min_lag72,wind_speed_std_lag72,year,month,weekend,day,hour,log_square_feet
0,105,0,1,0,50623,3.8,0.0,2.4,0.0,1021.0,...,240.0,3.1,3.1,0.0,2016,1,4,1,0,10.836
1,106,0,1,0,5374,3.8,0.0,2.4,0.0,1021.0,...,240.0,3.1,3.1,0.0,2016,1,4,1,0,8.586
2,106,3,1,0,5374,3.8,0.0,2.4,0.0,1021.0,...,240.0,3.1,3.1,0.0,2016,1,4,1,0,8.586
3,107,0,1,0,97532,3.8,0.0,2.4,0.0,1021.0,...,240.0,3.1,3.1,0.0,2016,1,4,1,0,11.484
4,108,0,1,0,81580,3.8,0.0,2.4,0.0,1021.0,...,240.0,3.1,3.1,0.0,2016,1,4,1,0,11.310


### Architecture 1 :-  Sequential Model with 2 Dense Layers

In [135]:
Batch_Size = 64
Learning_rate = 0.0003
Epochs = 30

In [136]:
model_reg = Sequential()
model_reg.add(Dense(256, kernel_initializer='glorot_normal', activation='relu', input_dim=40))
model_reg.add(Dense(128, kernel_initializer='glorot_normal', activation='relu'))
model_reg.add(Dense(1))

In [141]:
adam = optimizers.adam(lr=Learning_rate)
model_reg.compile(loss='mse', optimizer=adam)

In [142]:
model_reg.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 256)               10496     
_________________________________________________________________
dense_46 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_47 (Dense)             (None, 1)                 129       
Total params: 43,521
Trainable params: 43,521
Non-trainable params: 0
_________________________________________________________________


In [143]:
history = model_reg.fit(x_train, target,
                    epochs=Epochs,
                    verbose=1, 
                    validation_data=(x_test, target_test),
                    )

Train on 1331391 samples, validate on 570597 samples
Epoch 1/30
1331391/1331391 [==============================] - 133s 100us/step - loss: 4.1136 - val_loss: 4.3162
Epoch 2/30
1331391/1331391 [==============================] - 132s 99us/step - loss: 3.6761 - val_loss: 3.5274
Epoch 3/30
1331391/1331391 [==============================] - 133s 100us/step - loss: 3.5903 - val_loss: 3.4059
Epoch 4/30
1331391/1331391 [==============================] - 129s 97us/step - loss: 3.5544 - val_loss: 3.3114
Epoch 5/30
1331391/1331391 [==============================] - 127s 95us/step - loss: 3.5271 - val_loss: 3.3373
Epoch 6/30
1331391/1331391 [==============================] - 123s 92us/step - loss: 3.4890 - val_loss: 3.3125
Epoch 7/30
1331391/1331391 [==============================] - 123s 93us/step - loss: 3.4544 - val_loss: 3.2510
Epoch 8/30
1331391/1331391 [==============================] - 124s 93us/step - loss: 3.4302 - val_loss: 3.4877
Epoch 9/30
1331391/1331391 [=============================

In [144]:
''"This is a scorer function for calaculating RMSLE"''

def rmsle_score(y, y_pred):
    assert len(y) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5

In [160]:
print('Train RMSLE = ',rmsle_score(np.expm1(target) , np.expm1(model_reg.predict(x_train))))

Train RMSLE =  1.6063063324663285


In [145]:
preds = np.expm1(model_reg.predict(x_test,batch_size = 64,verbose = 1))
print('RMSLE of Test set is ',rmsle_score(np.expm1(target_test).values , preds))

570597/570597 [==============================] - 9s 15us/step
RMSLE of Test set is  1.6111324618429952


### Architecture 2  :-  Sequential Model with 6 Dense Layers 

In [148]:
tr_input = x_train.shape[1] 
te_input = x_test.shape[1]

In [153]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Flatten, BatchNormalization


# create the  network
model = Sequential()
model.add(Dense(512, activation='relu',   input_dim=tr_input))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(1, activation='relu'))
model.compile(loss='mse', optimizer='adam')
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 512)               20992     
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 512)               2048      
_________________________________________________________________
dense_61 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_17 (Batc (None, 256)               1024      
_________________________________________________________________
dense_62 (Dense)             (None, 128)             

In [157]:
model.fit(x_train, target,  batch_size = 4168,
          epochs = 60, verbose = 1, 
          validation_data=(x_test, target_test))

Train on 1331391 samples, validate on 570597 samples
Epoch 1/60
1331391/1331391 [==============================] - 10s 7us/step - loss: 6.2917 - val_loss: 3.8786TA: 1s 
Epoch 2/60
1331391/1331391 [==============================] - 8s 6us/step - loss: 4.3589 - val_loss: 3.6000
Epoch 3/60
1331391/1331391 [==============================] - 8s 6us/step - loss: 4.0567 - val_loss: 3.6465
Epoch 4/60
1331391/1331391 [==============================] - 8s 6us/step - loss: 3.9569 - val_loss: 3.6188
Epoch 5/60
1331391/1331391 [==============================] - 8s 6us/step - loss: 3.9008 - val_loss: 3.5783
Epoch 6/60
1331391/1331391 [==============================] - 8s 6us/step - loss: 3.8707 - val_loss: 3.6389
Epoch 7/60
1331391/1331391 [==============================] - 8s 6us/step - loss: 3.8365 - val_loss: 3.5075
Epoch 8/60
1331391/1331391 [==============================] - 8s 6us/step - loss: 3.8112 - val_loss: 3.4478
Epoch 9/60
1331391/1331391 [==============================] - 8s 6us/step -

In [161]:
print('Train RMSLE = ',rmsle_score(np.expm1(target) , np.expm1(model.predict(x_train))))

Train RMSLE =  1.893043048351508


In [158]:
preds = np.expm1(model.predict(x_test,batch_size = 4168,verbose = 1))
print('RMSLE of Test set is ',rmsle_score(np.expm1(target_test).values , preds))

570597/570597 [==============================] - 1s 2us/step
RMSLE of Test set is  1.8631497456782922
